## How should I use truth table as embedding, where the output can be in witin the input?


A single row may look like:
<table>
  <tr>
    <th>Latch example</th>
    <th colspan="4">Input</th>
    <th colspan="2">Output</th>
  </tr>
  <tr>
    <td>Value</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>0</td>
    <td>1</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Variable name</td>
    <td>inp0</td>
    <td>inp1</td>
    <td>out0</td>
    <td>out1</td>
    <td>out0</td>
    <td>out1</td>
  </tr>
</table>


For variable name, use categorical labels of size 2. It should be fine to just concatenate the values and names as they would both be binary.

Ahh, but it also needs a n/a for empty values

Maybe have the vector be (0, 1, N/A), and then the types are fixed by position. Make input have 16 values, and output have 8 values. That is up to 8 inputs and 8 outputs.


For embedding the entire table. Take all the row embeddings and use a transformer.

In [ ]:
import torch
import torch.nn as nn
from enum import Enum
import numpy as np

#What does nn.parameter do?

# I really do not like working with enums in Python. Making a truthtable where every element is
# SocketState.state.value is incredibly verbose, and why is there not a better way to infer using 
# the integer values!
# class SocketState(Enum):
#     OFF = 0
#     ON = 1
#     UNUSED = 2
OFF = 0
ON = 1
UNUSED = 2


class RowEmbedder(nn.Module):
    def __init__(self, num_categories, vector_length, embedding_dim):
        super().__init__()
        self.shared_embed = nn.Embedding(num_categories, embedding_dim)
        self.position_weights = nn.Parameter(torch.ones(vector_length, embedding_dim))
        self.position_bias = nn.Parameter(torch.zeros(vector_length, embedding_dim))
        
    def forward(self, x):
        # x shape: [batch_size, vector_length]
        shared = self.shared_embed(x)  # [batch_size, vector_length, emb_dim]
        # Apply position-specific scaling and shifting
        return shared * self.position_weights + self.position_bias


# (inp1, inp2) -> (out1)
ANDTable = torch.tensor([
    [ON, ON, ON],
    [OFF, ON, OFF],
    [OFF, OFF, OFF],
    [ON, OFF, OFF],  
])
# When i have 2 inputs, there are 2^2 = 4 possible combinations
# When i have 16 + 8 -> 24 coloumns, of which 16 are inputs. I would have 2^16 combinations. that would be 65536 rows per table...

# But often, some if not most of these would be UNUSED. 

testEmbedder = RowEmbedder(3, 3, 5)

rowsEmbeddded = testEmbedder(ANDTable) 
print(rowsEmbeddded.shape) 
rowsEmbeddded

torch.Size([4, 3, 5])


tensor([[[ 0.1049,  1.5537,  0.3037,  0.0900,  0.8772],
         [ 0.1049,  1.5537,  0.3037,  0.0900,  0.8772],
         [ 0.1049,  1.5537,  0.3037,  0.0900,  0.8772]],

        [[-0.6277, -0.7029,  0.8341,  0.5222,  1.4537],
         [ 0.1049,  1.5537,  0.3037,  0.0900,  0.8772],
         [-0.6277, -0.7029,  0.8341,  0.5222,  1.4537]],

        [[-0.6277, -0.7029,  0.8341,  0.5222,  1.4537],
         [-0.6277, -0.7029,  0.8341,  0.5222,  1.4537],
         [-0.6277, -0.7029,  0.8341,  0.5222,  1.4537]],

        [[ 0.1049,  1.5537,  0.3037,  0.0900,  0.8772],
         [-0.6277, -0.7029,  0.8341,  0.5222,  1.4537],
         [-0.6277, -0.7029,  0.8341,  0.5222,  1.4537]]],
       grad_fn=<AddBackward0>)

In [24]:
# Tabluar encoder

class TabularTransformer(nn.Module):
    def __init__(self, num_features, num_categories, d_model):
        super().__init__()
        self.d_model = d_model

        self.row_embedding = RowEmbedder(num_categories, num_features, d_model) #num_categories, vector_length, embedding_dim

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=8,
            dim_feedforward=4*d_model,
            batch_first=True,
        )

        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=6,
        )
    
    def forward(self, x):
        batch_size, num_rows, num_cols = x.shape

        rows = self.row_embedding(x) #bs, rows, columns, embedding
        rows = rows.mean(dim=2)
        
        print(rows.shape)
        transformed = self.transformer(rows)
        print(transformed.shape)

        return transformed

tester = TabularTransformer(3, 3, 16)
tester(ANDTable.unsqueeze(0))

torch.Size([1, 4, 16])
torch.Size([1, 4, 16])


tensor([[[-0.1436, -0.6812, -0.0235,  0.6048, -0.0848,  0.0645, -1.0959,
          -1.2769,  0.6558,  0.8663,  1.8017, -0.6553, -2.0930, -0.1557,
           0.5661,  1.6507],
         [-0.0702, -0.6543,  0.0652,  0.2972, -0.6605,  0.5285, -1.1609,
          -1.2212,  0.3690,  0.6083,  1.8073, -0.4109, -1.6503, -0.7006,
           0.8382,  2.0152],
         [-0.0956, -0.5989,  0.2707, -0.3734, -0.2228,  0.3948, -1.2477,
          -1.2040,  0.3847,  1.0032,  2.0413, -0.2551, -1.8056, -0.5681,
           0.5696,  1.7069],
         [ 0.3364, -0.6442,  0.1417,  0.6431, -0.6152,  0.3515, -1.3678,
          -1.4023,  0.4750,  0.9910,  1.7164, -0.3414, -1.8087, -0.5382,
           0.4230,  1.6396]]], grad_fn=<NativeLayerNormBackward0>)

In [30]:
ANDFullTable = torch.tensor([
    [ON, ON, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, ON, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED],
    [OFF, ON, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, OFF, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED],
    [OFF, OFF, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, OFF, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED],
    [ON, OFF, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, OFF, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED, UNUSED],  
])

tableTransformer = TabularTransformer(24, 3, 16)
tableTransformer(ANDFullTable.unsqueeze(0))#.mean(dim=1).shape

torch.Size([1, 4, 16])
torch.Size([1, 4, 16])


tensor([[[ 1.0100,  0.9726,  0.3281, -0.7896,  0.8628, -0.7175, -2.1203,
           0.0937,  1.6237, -0.6103,  0.5341,  0.6800,  0.9845, -0.9447,
          -1.1796, -0.7275],
         [ 1.1550,  1.4748,  0.2274, -0.9185,  1.1520, -0.8207, -1.7552,
           0.4032,  1.3018, -0.8060,  0.6085,  0.4700,  0.5520, -1.0734,
          -1.0766, -0.8944],
         [ 1.2859,  0.9271,  0.5452, -0.7221,  0.8489, -0.7952, -2.1205,
           0.4179,  1.6112, -0.5901,  0.4462,  0.4519,  0.6022, -1.2450,
          -0.8267, -0.8369],
         [ 0.9696,  1.3358,  0.6448, -0.8679,  0.8405, -0.1166, -1.6423,
           0.0502,  1.2079, -0.9004,  0.6790,  0.6995,  0.8512, -1.2895,
          -1.3870, -1.0750]]], grad_fn=<NativeLayerNormBackward0>)